In [1]:
import numpy as np
from kgcnn.data.datasets.MD17RevisedDataset import MD17RevisedDataset

2022-12-01 17:29:24.324886: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-01 17:29:24.410245: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-01 17:29:24.413031: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-01 17:29:24.413040: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [2]:
dataset = MD17RevisedDataset("aspirin")
print("Number of steps", len(dataset))

INFO:kgcnn.data.download:Checking and possibly downloading dataset with name MD17Revised
INFO:kgcnn.data.download:Dataset directory located at /home/patrickreiser/.kgcnn/datasets
INFO:kgcnn.data.download:Dataset directory found. Done.
INFO:kgcnn.data.download:Dataset found. Done.
INFO:kgcnn.data.download:Directory for extraction exists. Done.
INFO:kgcnn.data.download:Not extracting tar File. Stopped.


Number of steps 100000


In [3]:
data = dataset[:500]
data.map_list(method="set_range", node_coordinates="coords")
data[0].keys()

dict_keys(['coords', 'energies', 'forces', 'old_indices', 'old_energies', 'old_forces', 'nuclear_charges', 'range_indices', 'range_attributes'])

In [4]:
eng = np.expand_dims(data.get("energies"), axis=-1)
eng.shape, eng[:3]

((500, 1),
 array([[-406276.78237402],
        [-406272.40203419],
        [-406282.16488753]]))

In [5]:
forces = data.tensor({"name": "forces", "ragged": True, "shape": (None, 3)})
atoms = data.get("nuclear_charges")
forces.shape

2022-12-01 17:29:28.446296: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-01 17:29:28.446313: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-01 17:29:28.446326: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (patrick-tuxedoinfinitybookpro14gen6): /proc/driver/nvidia/version does not exist
2022-12-01 17:29:28.446567: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


TensorShape([500, None, 3])

In [6]:
from kgcnn.scaler.force import EnergyForceExtensiveScaler
scaler = EnergyForceExtensiveScaler(standardize_coordinates=False, standardize_scale=False)  # For testing no scale.
_, eng, _ = scaler.fit_transform(X=None, y=eng, force=forces, atomic_number=atoms)
eng[:3]

array([[-2.12796113],
       [ 2.25237869],
       [-7.51047464]])

In [7]:
from kgcnn.model.force import EnergyForceModel

In [8]:
model = EnergyForceModel(
    module_name="kgcnn.literature.Schnet", 
    class_name="make_model", 
    output_as_dict=True,
    config={
        "name": "Schnet",
        "inputs": [{"shape": (None,), "name": "nuclear_charges", "dtype": "float32", "ragged": True},
                   {"shape": (None, 3), "name": "coords", "dtype": "float32", "ragged": True},
                   {"shape": (None, 2), "name": "range_indices", "dtype": "int64", "ragged": True}],
        "input_embedding": {"node": {"input_dim": 95, "output_dim": 64}},
        "make_distance": True, "expand_distance": True,
        "interaction_args": {"units": 128, "use_bias": True,
                             "activation": "kgcnn>shifted_softplus", "cfconv_pool": "sum"},
        "node_pooling_args": {"pooling_method": "sum"},
        "depth": 4,
        "gauss_args": {"bins": 20, "distance": 4, "offset": 0.0, "sigma": 0.4},
        "verbose": 10,
        "last_mlp": {"use_bias": [True, True], "units": [128, 64],
                     "activation": ["kgcnn>shifted_softplus", "kgcnn>shifted_softplus"]},
        "output_embedding": "graph", "output_to_tensor": True,
        "use_output_mlp": True,
        "output_mlp": {"use_bias": [True, True], "units": [64, 1],
                       "activation": ["kgcnn>shifted_softplus", "linear"]}},
    output_to_tensor=False,
    output_squeeze_states=True,
)

INFO:kgcnn.model.utils:Updated model kwargs:
INFO:kgcnn.model.utils:{'name': 'Schnet', 'inputs': ListWrapper([DictWrapper({'shape': (None,), 'name': 'nuclear_charges', 'dtype': 'float32', 'ragged': True}), DictWrapper({'shape': (None, 3), 'name': 'coords', 'dtype': 'float32', 'ragged': True}), DictWrapper({'shape': (None, 2), 'name': 'range_indices', 'dtype': 'int64', 'ragged': True})]), 'input_embedding': {'node': {'input_dim': 95, 'output_dim': 64}}, 'make_distance': True, 'expand_distance': True, 'interaction_args': {'units': 128, 'use_bias': True, 'activation': 'kgcnn>shifted_softplus', 'cfconv_pool': 'sum'}, 'node_pooling_args': {'pooling_method': 'sum'}, 'depth': 4, 'gauss_args': {'bins': 20, 'distance': 4, 'offset': 0.0, 'sigma': 0.4}, 'verbose': 10, 'last_mlp': {'use_bias': ListWrapper([True, True]), 'units': ListWrapper([128, 64]), 'activation': ListWrapper(['kgcnn>shifted_softplus', 'kgcnn>shifted_softplus'])}, 'output_embedding': 'graph', 'output_to_tensor': True, 'use_outpu

In [9]:
x_tensor = data.tensor(model.model_config["inputs"])
print([x.shape for x in x_tensor])

[TensorShape([500, None]), TensorShape([500, None, 3]), TensorShape([500, None, 2])]


In [10]:
test_out = model.predict(x_tensor)

16/16 [==============================] - 3s 31ms/step


In [11]:
[i.shape for i in test_out.values()]

[(500, 1), TensorShape([500, None, 3])]

In [12]:
from kgcnn.metrics.loss import RaggedMeanAbsoluteError
from tensorflow.keras.optimizers import Adam

In [13]:
model.compile(
    loss={"energy": "mean_absolute_error", "force": RaggedMeanAbsoluteError()},
    optimizer=Adam(learning_rate=5e-04),
    metrics=None,
    loss_weights=[1, 20],
)

hist = model.fit(
    x_tensor, {"energy": eng, "force": forces},
    shuffle=True,
    batch_size=64,
    epochs=1500,
    verbose=2,
)

Epoch 1/1500
8/8 - 8s - loss: 440.4033 - energy_loss: 4.7874 - force_loss: 21.7808 - 8s/epoch - 949ms/step
Epoch 2/1500
8/8 - 1s - loss: 440.2900 - energy_loss: 4.8008 - force_loss: 21.7745 - 1s/epoch - 161ms/step
Epoch 3/1500
8/8 - 1s - loss: 439.0919 - energy_loss: 5.0521 - force_loss: 21.7020 - 1s/epoch - 143ms/step
Epoch 4/1500
8/8 - 1s - loss: 437.1944 - energy_loss: 11.0558 - force_loss: 21.3069 - 1s/epoch - 149ms/step
Epoch 5/1500
8/8 - 1s - loss: 435.4678 - energy_loss: 9.3724 - force_loss: 21.3048 - 1s/epoch - 141ms/step
Epoch 6/1500
8/8 - 1s - loss: 432.5701 - energy_loss: 5.9192 - force_loss: 21.3325 - 1s/epoch - 158ms/step
Epoch 7/1500
8/8 - 1s - loss: 424.6663 - energy_loss: 8.0991 - force_loss: 20.8284 - 1s/epoch - 141ms/step
Epoch 8/1500
8/8 - 1s - loss: 417.2795 - energy_loss: 15.7604 - force_loss: 20.0760 - 1s/epoch - 141ms/step
Epoch 9/1500
8/8 - 1s - loss: 404.5482 - energy_loss: 14.1636 - force_loss: 19.5192 - 1s/epoch - 148ms/step
Epoch 10/1500
8/8 - 1s - loss: 387

KeyboardInterrupt: 

In [ ]:
from kgcnn.utils.plots import plot_train_test_loss

In [ ]:
plot_train_test_loss([hist]);